#### Mapping a Class against Arbitrary Subqueries

Similar to _mapping against a join_, a plain `select()` object __can be used with a mapper__ as well. The example fragment below illustrates mapping a class called _Customer_ to a `select()` which __includes a join to a subquery__.

In [3]:
from sqlalchemy import Column, ForeignKey, Integer, String, Table, select, func
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [4]:
customers = Table(
    "customers",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [5]:
orders = Table(
    "orders",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("customer_id", Integer, ForeignKey("customers.id")),
    Column("price", Integer),
)

In [6]:
subq = (
    select(
        func.count(orders.c.id).label("order_count"),
        func.max(orders.c.price).label("highest_order"),
        orders.c.customer_id,
    ).group_by(orders.c.customer_id).subquery()
)

In [7]:
customer_select = (
    select(customers, subq).
    join_from(customers, subq, customers.c.id == subq.c.customer_id).
    subquery()
)

In [8]:
class Customer(Base):
    __table__ = customer_select

Above, the full row represented by *customer_select* will be _all the columns of the customers table_, in addition to those __columns exposed by the `subq` subquery__, which are *order_count*, *highest_order*, and *customer_id*. Mapping the _Customer_ class to this _selectable_ then __creates a class which will contain those attributes__.

When the _ORM_ __persists__ _new instances_ of `Customer`, __only the customers table will actually receive an INSERT__. This is because the _primary key_ of the _orders_ table is __not represented in the mapping__; the _ORM_ will __only emit an INSERT__ into a table for which it has _mapped the primary key_.

> ##### Note
>
> The practice of _mapping to arbitrary SELECT statements_, especially _complex ones_ as above, is __almost never needed__; it _necessarily_ tends to produce _complex queries_ which are __often less efficient__ than that which would be produced by _direct query construction_. The practice is to some degree based on the very early history of SQLAlchemy where the `mapper()` construct was meant to _represent the primary querying interface_; in modern usage, the _Query_ object can be used to __construct virtually any SELECT statement__, including _complex composites_, and __should be favored__ over the `"map-to-selectable" approach`.